### Exercice 1: Calculer la moyenne de la puissance administrative (puiss_admin_98) pour chaque marque de voiture (lib_mrq).



In [62]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()
df = spark.read.csv('../data/car.csv', header=True, inferSchema=True, sep=';')

In [63]:
df.groupby(df.lib_mrq)\
  .agg(
    round(avg(df.puiss_admin_98), 2).alias('avg_puiss_admin_98')
  )\
  .show()

+-----------+------------------+
|    lib_mrq|avg_puiss_admin_98|
+-----------+------------------+
|   MERCEDES|             10.99|
|LAMBORGHINI|             53.39|
|    HYUNDAI|              6.94|
|      DACIA|              5.14|
|       FIAT|               6.3|
|    CITROEN|              6.47|
|    BENTLEY|             48.15|
|       LADA|               6.0|
|   INFINITI|             17.69|
|       AUDI|             12.92|
|       FORD|              7.41|
|    FERRARI|             52.44|
|  CHEVROLET|             14.43|
|     JAGUAR|             26.67|
| ALFA-ROMEO|               7.8|
|      LEXUS|             18.83|
|       JEEP|             16.43|
|      MAZDA|              8.07|
|        BMW|             14.36|
|   MASERATI|             29.08|
+-----------+------------------+
only showing top 20 rows



### Exercice 2: Trouver la voiture ayant la consommation mixte (conso_mixte) la plus basse.



In [64]:
min_conso_mixte = df.agg(min(df.conso_mixte)).collect()[0][0]
df.filter(df.conso_mixte == min_conso_mixte)\
  .collect()[0].asDict()

{'lib_mrq': 'BMW',
 'lib_mod_doss': 'SERIE I',
 'lib_mod': 'I3',
 'dscom': "i3 avec Prolongateur d'autonomie",
 'cnit': 'M10BM2VP0000008',
 'tvv': 'BMWi-11Z416A040000',
 'cod_cbr': 'EE',
 'hybride': 'oui',
 'puiss_admin_98': 3,
 'puiss_max': '28',
 'typ_boite_nb_rapp': 'V 0',
 'conso_urb': None,
 'conso_exurb': None,
 'conso_mixte': '0,600000024',
 'co2': 13,
 'co_typ_1': '0,052999999',
 'hc': '0,004',
 'nox': '0,002',
 'hcnox': None,
 'ptcl': None,
 'masse_ordma_min': 1390,
 'masse_ordma_max': 1390,
 'champ_v9': '715/2007*195/2013EURO6',
 'date_maj': 'mars-14',
 'Carrosserie': 'BERLINE',
 'gamme': 'INFERIEURE',
 '_c26': None,
 '_c27': None,
 '_c28': None,
 '_c29': None}

### Exercice 3: Filtrer les voitures dont la puissance maximale (puiss_max) est supérieure à 200 chevaux.



In [65]:
df.filter(df.puiss_max > 200)\
  .count()

1781

### Exercice 4: Calculer le nombre de voitures hybrides par carrosserie.



In [66]:
df.filter(df.hybride == 'oui')\
  .groupby(df.Carrosserie, df.hybride)\
  .count()\
  .show()

+-------------------+-------+-----+
|        Carrosserie|hybride|count|
+-------------------+-------+-----+
|          MINISPACE|    oui|    1|
|              BREAK|    oui|   11|
|TS TERRAINS/CHEMINS|    oui|   47|
|              COUPE|    oui|    5|
|            BERLINE|    oui|  325|
+-------------------+-------+-----+



### Exercice 5: Calculer le pourcentage de voitures émettant moins de 120 g/km de CO<sub>2</sub> (co2) par gamme de voiture.



In [67]:
df.groupby(df.gamme).count()\
  .withColumnRenamed('count', 'nb_total')\
  .join(
    df.filter(df.co2 < 120)\
      .groupby(df.gamme).count()\
      .withColumnRenamed('count', 'nb_cars_co2_lt_120'),
    on='gamme'
  ).withColumn('percentage_of_total', 
              round(
                  col('nb_cars_co2_lt_120') / col('nb_total') * 100,
                  2
              ))\
  .show()

+----------+--------+------------------+-------------------+
|     gamme|nb_total|nb_cars_co2_lt_120|percentage_of_total|
+----------+--------+------------------+-------------------+
| MOY-INFER|   33438|               762|               2.28|
|SUPERIEURE|    1513|               156|              10.31|
|ECONOMIQUE|     269|               171|              63.57|
|      LUXE|    3597|               112|               3.11|
|INFERIEURE|    1073|               473|              44.08|
| MOY-SUPER|   15098|               148|               0.98|
+----------+--------+------------------+-------------------+



### Exercice 6: Trouver la carrosserie la plus courante parmi les voitures de chaque gamme.



In [68]:
df = df.withColumn("gamme", regexp_replace(col("gamme"), 'MOY-INFERIEURE', 'MOY-INFER'))

In [69]:
df.groupby('gamme', 'Carrosserie')\
  .agg(count('*').alias('count'))\
  .orderBy('gamme', col('count').desc())\
  .groupby('gamme')\
  .agg(first("Carrosserie").alias('top_carrosserie_de_la_gamme'), first('count').alias('count')) \
  .show()

+----------+---------------------------+-----+
|     gamme|top_carrosserie_de_la_gamme|count|
+----------+---------------------------+-----+
|ECONOMIQUE|                    BERLINE|  179|
|INFERIEURE|                    BERLINE|  542|
|      LUXE|                    BERLINE| 1615|
| MOY-INFER|                    MINIBUS|31828|
| MOY-SUPER|                    MINIBUS|14279|
|SUPERIEURE|                    BERLINE|  799|
+----------+---------------------------+-----+



### Exercice 7: Calculer la différence entre la puissance maximale (puiss_max) et la puissance administrative (puiss_admin_98) pour chaque voiture.



In [93]:
df.withColumn('diff_puissance', col('puiss_max').astype('int') - col('puiss_admin_98'))\
  .select('puiss_max', 'puiss_admin_98', 'diff_puissance')\
  .show()

+---------+--------------+--------------+
|puiss_max|puiss_admin_98|diff_puissance|
+---------+--------------+--------------+
|      147|            12|           135|
|      147|            12|           135|
|      100|             7|            93|
|      100|             7|            93|
|      125|             9|           116|
|      125|             9|           116|
|      125|             9|           116|
|      100|             7|            93|
|      100|             7|            93|
|      125|             9|           116|
|      125|             9|           116|
|      125|             9|           116|
|      177|            14|           163|
|      331|            38|           293|
|      331|            38|           293|
|      147|            12|           135|
|      147|            12|           135|
|      125|             9|           116|
|      125|             9|           116|
|      120|             9|           111|
+---------+--------------+--------

### Exercice 8: Trouver les marques de voiture (lib_mrq) dont la consommation urbaine (conso_urb) est inférieure à la consommation mixte (conso_mixte).

In [94]:
df.filter(col('conso_urb').cast('float') < col('conso_mixte').cast('float'))\
  .select('lib_mrq', 'conso_urb', 'conso_mixte')\
  .show()

+-------+---------+-----------+
|lib_mrq|conso_urb|conso_mixte|
+-------+---------+-----------+
+-------+---------+-----------+



### Exercice 9: Calculer la moyenne des émissions de CO<sub>2</sub> (co2) par année de mise à jour (annee_maj).



In [98]:
df.withColumn('annee_maj', 2000 + col('date_maj').substr(-2, 2).astype('int'))\
  .select('annee_maj', 'co2')\
  .groupby('annee_maj')\
  .agg(round(avg('co2'), 2).alias('avg_co2'))\
  .show()

+---------+-------+
|annee_maj|avg_co2|
+---------+-------+
|     2013|  189.3|
|     null| 203.78|
|     2014| 165.15|
+---------+-------+



### Exercice 10: Trouver les voitures dont la masse_ordma_max est supérieure à la moyenne de la masse_ordma_max de toutes les voitures.

In [112]:
avg_masse_ordma_max = df.agg(avg('masse_ordma_max')).collect()[0][0]
df.filter(col('masse_ordma_max') > avg_masse_ordma_max)\
  .count()

29413